<a href="https://colab.research.google.com/github/Steve-YJ/Extracting-Important-Feature-of-Malimg-using-VAE/blob/master/02_Training_MNIST_AAE_Parameter_Tunning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 02. Training MNIST_AAE_

1st. Run AAE Architecture on MNIST dataset by learning training process

## Reference
* god....!: https://github.com/eriklindernoren/PyTorch-GAN/blob/master/implementations/aae/aae.py

### What I want to do
from 20.07.15.wed<br>
<br>

* run AAE -> Done
* practicing train -> Do more
* practice & make my own work flow -> Make Template<br>
    * continue: https://github.com/Steve-YJ/Colab_Exercise/blob/master/Again_Training_Exp05_Just_20Epochs.ipynb

---
### Log
1. 20.07.15. Train 200 Epochs <- Original
2. 20.07.16. Change Parameters & Train 200 Epochs

---
Start it<br><br><br><br><br>

## 00. Mount Drive

* If you use colab, first mount driver

First, always check if your GPU is possible or not


In [ ]:
! nvidia-smi

Thu Jul 16 08:47:35 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.51.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   35C    P0    28W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

set 'autoreload'

In [ ]:
%load_ext autoreload
%autoreload 2

Mount Drive

In [ ]:
# mount drive(Drive Mount)

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
! pwd

/content


In [ ]:
# %cd drive/My\ Drive/Post_InfoSec_Exp
%cd drive/My\ Drive/Post_InfoSec_Exp_200715
! pwd

/content/drive/My Drive/Post_InfoSec_Exp_20.07.15(wed~)
/content/drive/My Drive/Post_InfoSec_Exp_20.07.15(wed~)


## 01. Library Import

In [ ]:
import argparse
import os
import numpy as np
import math
import itertools  # what is this?

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch


# 데이터 전처리는 주로 torchvision에서 하는데?
# e.g torchivision.transforms, torchivision.save_image, torchivision.datasets

# 직접적으로 train과 관련된 부분은 torch를 활용한다
# torch.utils.data DataLoader, torch.autograd Variable, torch, torch.nn, torch.nn.functional

In [ ]:
# os library를 사용해 directory를 만드는 방법을 배워보자

# os.makedirs("Exp02_images", exist_ok=True)
os.makedirs("Exp02_images", exist_ok=True)

In [ ]:
# Image shape를 정하고 간다
# 1channel 32 width 32 height

img_shape = (1, 32, 32)

cuda = True if torch.cuda.is_available() else False

Q. 모델의 파라미터를 어디에서 정해주면 좋을까?<br>

A1. Library Import 단계에서?<br>
A2. Training을 시작하기 전?(<- 후자가 좀 더 낫지 않을까?)

## 02. Data Preprocessing
* Load dataset
* preprocess it
    * transforms
    * make custom dataset
    * train_test split
    * make train loader, test loader

* work Flow
    * transforms module 사용해서 image data compose
        * size 조정 및 normalize, tensor 변환
        * Image Folder를 이용해 dataload
        * dataset split: train dataset, test dataset
        * DataLoader로 batch단위 dataset 불러오기

### 2-1. Configure data loader
* 본 Tutorial에서는 Data Loader에 대해서만 신경써주면 될 것 같다
<br>
<br>


Parameter settings<br>
* for preprocessing
    * transforms.Resize: 32
    * transforms.Normalize: 0.5(mean), 0.5(std)
* for Data Loader
    * batch_size: 64
    * shuffle: True(shuffle it)


In [ ]:
# configure data loader

os.makedirs("./data/mnist", exist_ok=True)  # make directory: ./data/mnist

# Define DataLoader: Data loader. Combines a dataset and a sampler, and provides an iterable over the given dataset. 
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST(
        "./data/mnist",  # root
        train=True,
        download=True,
        transform = transforms.Compose(
            [transforms.Resize(32),  # Resizing
             transforms.ToTensor(),  # convert numpy to tensor
             transforms.Normalize([0.5], [0.5])]  # transforms.Normalize(): Normalize a tensor image with mean and standard deviation.
        ),
    ),
    batch_size=64,
    shuffle=True
)  

# 03. Define Adversarial Variational AutoEncoder Class

* Encoder Class
* Decoder Class
* Discriminator Class
* Optimizer

### reparameterization module

In [ ]:
def reparameterization(mu, logvar):
    std = torch.exp(logvar / 2)
    sampled_z = Variable(Tensor(np.random.normal(0, 1, (mu.size(0), 10))))
    z = sampled_z * std + mu  # paper의 수식과 동일
    return z

### 3-1. Encoder Class

In [ ]:
class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(int(np.prod(img_shape)), 512),  # prod == product
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            nn.LeakyReLU(0.2, inplace=True),
        )

        self.mu = nn.Linear(512, 10)      # 10: latent dim
        self.logvar = nn.Linear(512, 10)  # 10: latent dim

    def forward(self, img):
        img_flat = img.view(img.shape[0], -1)  # 1x (32*32)
        x = self.model(img_flat)  # extract_vector: (batch_size, 10)
        mu = self.mu(x)
        logvar = self.logvar(x)
        z = reparameterization(mu, logvar)
        return z 

### 3-2. Decoder

In [ ]:
class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, int(np.prod(img_shape))),
            nn.Tanh(),
        )

    def forward(self, z):
        img_flat = self.model(z)
        img = img_flat.view(img_flat.shape[0], *img_shape)
        return img

### 3-3. Discriminator

In [ ]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        self.model = nn.Sequential(
            nn.Linear(10, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid(),
        )

    def forward(self, z):
        validity = self.model(z)
        return validity

### Define Loss Function

In [ ]:
# Use binary cross-entropy loss
adversarial_loss = torch.nn.BCELoss()
pixelwise_loss = torch.nn.L1Loss()

### Initialize Generator & Discriminator



In [ ]:
# Initialize generator and discriminator
encoder = Encoder()
decoder = Decoder()
discriminator = Discriminator()

if cuda:
    encoder.cuda()
    decoder.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    pixelwise_loss.cuda()

In [ ]:
discriminator.parameters()

<generator object Module.parameters at 0x7f1e98a39bf8>

### Optimizer

In [ ]:
optimizer_G = torch.optim.Adam(
    itertools.chain(encoder.parameters(), decoder.parameters()), lr=0.0002, betas=(0.5, 0.999)
)

# reduce optimizer_D's lr 0.0002 to 0.0002/50
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=0.0002/50, betas=(0.5, 0.999))

Tensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor

In [ ]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits"""
    # Sample noise
    z = Variable(Tensor(np.random.normal(0, 1, (n_row ** 2, 10))))
    gen_imgs = decoder(z)
    save_image(gen_imgs.data, "Exp02_images/%d.png" % batches_done, nrow=n_row, normalize=True)

## 04. Training

### Set parameters

* lr: learning rate
* 

In [ ]:
for epoch in range(200):
    for i, (imgs, _) in enumerate(dataloader):

        # Adversarial ground truths
        valid = Variable(Tensor(imgs.shape[0], 1).fill_(1.0), requires_grad=False)
        fake = Variable(Tensor(imgs.shape[0], 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(Tensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        encoded_imgs = encoder(real_imgs)
        decoded_imgs = decoder(encoded_imgs)

        # Loss measures generator's ability to fool the discriminator
        g_loss = 0.001 * adversarial_loss(discriminator(encoded_imgs), valid) + 0.999 * pixelwise_loss(
            decoded_imgs, real_imgs
        )

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Sample noise as discriminator ground truth
        z = Variable(Tensor(np.random.normal(0, 1, (imgs.shape[0], 10))))

        # Measure discriminator's ability to classify real from generated samples
        real_loss = adversarial_loss(discriminator(z), valid)
        fake_loss = adversarial_loss(discriminator(encoded_imgs.detach()), fake)
        d_loss = 0.5 * (real_loss + fake_loss)

        d_loss.backward()
        optimizer_D.step()

        print(
            "[Epoch %d/%d] [Batch %d/%d] [D loss: %f] [G loss: %f]"
            % (epoch, 200, i, len(dataloader), d_loss.item(), g_loss.item())
        )

        batches_done = epoch * len(dataloader) + i
        if batches_done % 400 == 0:
            sample_image(n_row=10, batches_done=batches_done)

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[Epoch 194/200] [Batch 629/938] [D loss: 0.430518] [G loss: 0.061357]
[Epoch 194/200] [Batch 630/938] [D loss: 0.453112] [G loss: 0.068540]
[Epoch 194/200] [Batch 631/938] [D loss: 0.446252] [G loss: 0.060231]
[Epoch 194/200] [Batch 632/938] [D loss: 0.419046] [G loss: 0.062710]
[Epoch 194/200] [Batch 633/938] [D loss: 0.483382] [G loss: 0.066914]
[Epoch 194/200] [Batch 634/938] [D loss: 0.402456] [G loss: 0.058093]
[Epoch 194/200] [Batch 635/938] [D loss: 0.495000] [G loss: 0.062533]
[Epoch 194/200] [Batch 636/938] [D loss: 0.513332] [G loss: 0.062164]
[Epoch 194/200] [Batch 637/938] [D loss: 0.388170] [G loss: 0.055355]
[Epoch 194/200] [Batch 638/938] [D loss: 0.433135] [G loss: 0.054198]
[Epoch 194/200] [Batch 639/938] [D loss: 0.499467] [G loss: 0.056569]
[Epoch 194/200] [Batch 640/938] [D loss: 0.419533] [G loss: 0.058189]
[Epoch 194/200] [Batch 641/938] [D loss: 0.427236] [G loss: 0.057504]
[Epoch 194/200] [Batch 642/938] [D loss: 0.562365] [G 